In [10]:
import numpy as np
import xgboost as xgb
import optuna
from optuna.visualization import plot_optimization_history
import pandas as pd 
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler

df = pd.read_csv("../data/processed/all_shots_final-v2.csv", index_col=0)
df.drop(['PLAYER1_NAME'], axis=1, inplace=True)
df.dropna(inplace=True)
df.columns

Index(['Shot Distance', 'Season Type', 'Shot Zone Basic_In The Paint (Non-RA)',
       'Shot Zone Basic_Right Corner 3', 'Shot Zone Area_Right Side(R)',
       'Shot Zone Range_8-16 ft.', 'at_home', 'PREVIOUS_OFF_MISSED', 'Age',
       'ASTM', 'ORBM', 'FT%', 'height', 'weight', 'C', 'SG-PG', 'E_DEF_RATING',
       'PCT_AREA', 'DETAILLED_SHOT_TYPE_JUMP SHOT', 'target'],
      dtype='object')

## Séparation des données

In [11]:
# prepare data for modeling   
X = df.drop('target', axis = 1)
y = df.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, stratify=y, shuffle=True)

## Standardisation et réduction de dimension

In [12]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

df_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
df_scaled['target'] = y_train

# réduction de dimensions
sel_var = VarianceThreshold(0.0001)
X_train_scaled = sel_var.fit_transform(X_train_scaled, y_train)
X_test_scaled = sel_var.transform(X_test_scaled)

## XGBoost

In [16]:
train = xgb.DMatrix(data=X_train_scaled, label=y_train)
test = xgb.DMatrix(data=X_test_scaled, label=y_test)


def objective(trial):
    param = {
        "verbosity": 0,
        "objective": "binary:logistic",
        # use exact for small dataset.
        "tree_method": "exact",
        # defines booster, gblinear for linear functions.
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
        # L2 regularization weight.
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        # L1 regularization weight.
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
        # sampling ratio for training data.
        "subsample": trial.suggest_float("subsample", 0.2, 1.0),
        # sampling according to each tree.
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.7, 1.0),
    }

    if param["booster"] in ["gbtree", "dart"]:
        # maximum depth of the tree, signifies complexity of the tree.
        param["max_depth"] = trial.suggest_int("max_depth", 3, 9, step=2)
        # minimum child weight, larger the term more conservative the tree.
        param["min_child_weight"] = trial.suggest_int("min_child_weight", 2, 10)
        param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
        # defines how selective algorithm is.
        param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])

    if param["booster"] == "dart":
        param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

    bst = xgb.train(param, train)
    preds = bst.predict(test)
    pred_labels = np.rint(preds)
    accuracy = accuracy_score(y_test, pred_labels)
    return accuracy

# Create a study object and optimize the objective function.
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2024-09-29 09:32:20,392] A new study created in memory with name: no-name-d4924156-25cc-48dd-82b9-510226349833
[I 2024-09-29 09:32:21,973] Trial 0 finished with value: 0.5645644595240594 and parameters: {'booster': 'dart', 'lambda': 0.00017663455485090256, 'alpha': 0.0011451390130396597, 'subsample': 0.6729488573361504, 'colsample_bytree': 0.7746800468691908, 'max_depth': 9, 'min_child_weight': 4, 'eta': 0.0006354985924249337, 'gamma': 1.0875921433686762e-05, 'grow_policy': 'lossguide', 'sample_type': 'weighted', 'normalize_type': 'forest', 'rate_drop': 0.012235251293054293, 'skip_drop': 0.0008846827664908837}. Best is trial 0 with value: 0.5645644595240594.
[I 2024-09-29 09:32:22,163] Trial 1 finished with value: 0.6739305327223757 and parameters: {'booster': 'gblinear', 'lambda': 0.016589581779152967, 'alpha': 1.2094464151932313e-08, 'subsample': 0.8053680207424867, 'colsample_bytree': 0.7350641807360068}. Best is trial 1 with value: 0.6739305327223757.
[I 2024-09-29 09:32:22,331]

Number of finished trials:  100
Best trial:
  Value: 0.6825352408268914
  Params: 
    booster: dart
    lambda: 1.0097915738299128e-06
    alpha: 2.3445234888982956e-08
    subsample: 0.7152405139734603
    colsample_bytree: 0.8437252255677583
    max_depth: 9
    min_child_weight: 9
    eta: 0.2357930936747691
    gamma: 0.13557767476363425
    grow_policy: lossguide
    sample_type: uniform
    normalize_type: tree
    rate_drop: 1.2065021262266629e-08
    skip_drop: 3.1483383573651435e-05


In [17]:
optuna.visualization.plot_optimization_history(study)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'mode': 'markers',
              'name': 'Objective Value',
              'type': 'scatter',
              'x': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,
                    18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
                    34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49,
                    50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65,
                    66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81,
                    82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97,
                    98, 99],
              'y': [0.5645644595240594, 0.6739305327223757, 0.6740587871792181,
                    0.5645644595240594, 0.6752247367868761, 0.5645644595240594,
                    0.6765306003474529, 0.6740704466752947, 0.5645644595240594,
                    0.5645644595240594, 0.6770202991826694, 0.6774866790257326,
                    0.6781279513099444, 0.5645644595240594, 0.6653724626021663,
                    0.5645644595240594, 0.680191682115499, 0.5666748283139202,
                    0.5645644595240594, 0.676390686394534, 0.5645644595240594,
                    0.67872258560985, 0.6771485536395117, 0.6768687257336737,
                    0.5645644595240594, 0.678582671656931, 0.5645644595240594,
                    0.5645644595240594, 0.5645644595240594, 0.6816141406368417,
                    0.6810661443212425, 0.6802849580841116, 0.6812760152506209,
                    0.5677824804411953, 0.6807746569193279, 0.5645644595240594,
                    0.6814159292035398, 0.5645644595240594, 0.5841407534366365,
                    0.6739421922184523, 0.5645644595240594, 0.6796320263038231,
                    0.679911854209661, 0.6808212949036343, 0.6821271584642111,
                    0.6739305327223757, 0.5645644595240594, 0.6594610980913405,
                    0.5645644595240594, 0.6740937656674478, 0.6816840976133012,
                    0.681217717770238, 0.681077803817319, 0.6811827392820082,
                    0.6748516329124256, 0.5654272622337263, 0.6809378898644001,
                    0.6780696538295615, 0.6749099303928084, 0.6650226777198689,
                    0.6811011228094722, 0.6815791621486119, 0.6737789592733802,
                    0.6800984061468863, 0.6825002623386617, 0.5948674898270897,
                    0.6811011228094722, 0.6761225179847727, 0.6780696538295615,
                    0.6722282462951952, 0.5645644595240594, 0.6807629974232514,
                    0.682243753424977, 0.6825235813308149, 0.6821621369524409,
                    0.6781745892942507, 0.6825352408268914, 0.6822204344328238,
                    0.6022829293317943, 0.6824419648582788, 0.682465283850432,
                    0.6794571338626745, 0.6821737964485175, 0.6807513379271748,
                    0.6810661443212425, 0.5645644595240594, 0.6719950563736635,
                    0.5645644595240594, 0.6810428253290893, 0.6807979759114811,
                    0.6819755850152156, 0.6815325241643057, 0.6822320939289004,
                    0.6740587871792181, 0.6797253022724358, 0.6762974104259214,
                    0.6606970046754579, 0.5709888418622547, 0.6740238086909883,
                    0.6810894633133956]},
             {'mode': 'lines',
              'name': 'Best Value',
              'type': 'scatter',
              'x': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,
                    18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
                    34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49,
                    50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65,
                    66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81,
                    82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97,
                    98, 99],
              'y': [0.5645644595240594, 0.67393053

In [18]:
optuna.visualization.plot_param_importances(study)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'cliponaxis': False,
              'hovertemplate': [booster (CategoricalDistribution):
                                0.07726350893741019<extra></extra>, lambda
                                (FloatDistribution):
                                0.08462061923529836<extra></extra>, alpha
                                (FloatDistribution):
                                0.14080752882677103<extra></extra>,
                                colsample_bytree (FloatDistribution):
                                0.3191073061752919<extra></extra>, subsample
                                (FloatDistribution):
                                0.37820103682522854<extra></extra>],
              'name': 'Objective Value',
              'orientation': 'h',
              'text': [0.08, 0.08, 0.14, 0.32, 0.38],
              'textposition': 'outside',
              'type': 'bar',
              'x': [0.07726350893741019, 0.08462061923529836, 0.14080752882677103,
                    0.3191073061752919, 0.37820103682522854],
              'y': [booster, lambda, alpha, colsample_bytree, subsample]}],
    'layout': {'template': '...',
               'title': {'text': 'Hyperparameter Importances'},
               'xaxis': {'title': {'text': 'Hyperparameter Importance'}},
               'yaxis': {'title': {'text': 'Hyperparameter'}}}
})

In [19]:
optuna.visualization.plot_slice(study, params = ['subsample',
'lambda',
'colsample_bytree',
'alpha',
'booster'])

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'marker': {'color': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,
                                   14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25,
                                   26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37,
                                   38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49,
                                   50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61,
                                   62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73,
                                   74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
                                   86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97,
                                   98, 99],
                         'colorbar': {'title': {'text': 'Trial'}, 'x': 1.0, 'xpad': 40},
                         'colorscale': [[0.0, 'rgb(247,251,255)'], [0.125,
                                        'rgb(222,235,247)'], [0.25,
                                        'rgb(198,219,239)'], [0.375,
                                        'rgb(158,202,225)'], [0.5,
                                        'rgb(107,174,214)'], [0.625,
                                        'rgb(66,146,198)'], [0.75,
                                        'rgb(33,113,181)'], [0.875,
                                        'rgb(8,81,156)'], [1.0, 'rgb(8,48,107)']],
                         'line': {'color': 'Grey', 'width': 0.5},
                         'showscale': True},
              'mode': 'markers',
              'name': 'Feasible Trial',
              'showlegend': False,
              'type': 'scatter',
              'x': [0.0011451390130396597, 1.2094464151932313e-08,
                    7.061111413598189e-07, 0.8866006585339498,
                    0.006164340132814618, 0.0022740045685392596,
                    3.2983150813660233e-07, 2.429573252265458e-07,
                    0.0020032067995748117, 0.00041512579285493613,
                    8.484494515479239e-06, 8.133467560031941e-06,
                    4.8657089248510754e-06, 1.1918050898581142e-05,
                    2.644798662957866e-05, 2.064716865213716e-06,
                    0.0001232411444965965, 0.03490800333137301,
                    9.270992213192295e-05, 4.812562213961896e-08,
                    0.00016718852040888474, 2.7533301873336584e-06,
                    1.8903133367431702e-06, 4.8501951817287105e-05,
                    2.5118473269649554e-06, 4.7175536113536835e-08,
                    4.073338713093635e-08, 1.5045402111711732e-07,
                    1.2145308056461768e-08, 6.767137965115558e-07,
                    0.0004828040058966788, 0.00025827731240911876,
                    0.0004171063819378436, 0.000504920751396847,
                    0.012575181356249463, 0.04136538891336599,
                    0.017317697424105542, 0.555976897514931, 0.13702261256162124,
                    0.0010697165365388674, 0.004828512822077445,
                    0.019211904897836914, 0.00837989629988712,
                    0.0011955649476340995, 0.0007924451904003184,
                    0.00046253507462241353, 0.002244616987781639,
                    2.2633788744447686e-05, 0.0033060088333697643,
                    4.1680599242801316e-05, 0.0009095763311571298,
                    0.0006605483030042633, 0.0008081315679075477,
                    0.00021073559203864343, 6.90066761693756e-05,
                    0.002330407681615806, 0.06258513237908052,
                    8.722819568856754e-07, 0.0054991117098776385,
                    0.1469340932356078, 0.0014409851603062105,
                    1.5235310717410962e-05, 1.1110020818814702e-05,
                    0.00011431096125171484, 6.454856185869622e-07,
                    3.067275731930734e-07, 6.899874960215372e-07,
                    2.542182211286341e-05, 4.8468202980693394e-06,
                    1.1069793122543056e-06, 1.9144492860569445e-07,
                    0.00029125284704099657, 4.0462

In [39]:
params = {
    "booster": "dart",
    "lambda": 1.0097915738299128e-06,
    "alpha": 2.3445234888982956e-08,
    "subsample": 0.7152405139734603,
    "colsample_bytree": 0.8437252255677583,
    "max_depth": 9,
    "min_child_weight": 9,
    "eta": 0.2357930936747691,
    "gamma": 0.13557767476363425,
    "grow_policy": "lossguide",
    "sample_type": "uniform",
    "normalize_type": "tree",
    "rate_drop": 1.2065021262266629e-08,
    "skip_drop": 3.1483383573651435e-05
}
model_xgb = xgb.train(params, train)

In [40]:
# save model
model_xgb.save_model("../reports/trained_xgb.save")

## Random Forest

In [20]:
from sklearn.ensemble import RandomForestClassifier
from optuna.samplers import TPESampler


def objective_rf(trial):
    n_estimators = trial.suggest_int('n_estimators', 100, 500)
    max_depth = trial.suggest_int('max_depth', 2, 32)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 32)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 32)
    max_features = trial.suggest_categorical('max_features', ['sqrt', 'log2'])
    criterion = trial.suggest_categorical('criterion', ["gini", "entropy", "log_loss"])

    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        criterion=criterion
    )

    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

# callback to stop the trial early 
pruning_callback = optuna.pruners.MedianPruner()

# Create a study object and optimize the objective function.
study = optuna.create_study(direction='maximize', pruner=[pruning_callback], sampler=TPESampler())
study.optimize(objective_rf, n_trials=100)

print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2024-09-29 09:35:16,649] A new study created in memory with name: no-name-6f775522-6b0e-46b2-89e0-2fd03ab494de
[I 2024-09-29 09:36:03,131] Trial 0 finished with value: 0.6813926102113866 and parameters: {'n_estimators': 149, 'max_depth': 23, 'min_samples_split': 31, 'min_samples_leaf': 18, 'max_features': 'sqrt', 'criterion': 'gini'}. Best is trial 0 with value: 0.6813926102113866.
[I 2024-09-29 09:36:27,094] Trial 1 finished with value: 0.6769969801905161 and parameters: {'n_estimators': 177, 'max_depth': 5, 'min_samples_split': 9, 'min_samples_leaf': 16, 'max_features': 'log2', 'criterion': 'gini'}. Best is trial 0 with value: 0.6813926102113866.
[I 2024-09-29 09:38:47,590] Trial 2 finished with value: 0.681217717770238 and parameters: {'n_estimators': 397, 'max_depth': 32, 'min_samples_split': 30, 'min_samples_leaf': 16, 'max_features': 'sqrt', 'criterion': 'log_loss'}. Best is trial 0 with value: 0.6813926102113866.
[I 2024-09-29 09:40:31,878] Trial 3 finished with value: 0.6825

Number of finished trials:  100
Best trial:
  Value: 0.6830832371424908
  Params: 
    n_estimators: 311
    max_depth: 14
    min_samples_split: 23
    min_samples_leaf: 4
    max_features: log2
    criterion: gini


In [21]:
plot_optimization_history(study)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'mode': 'markers',
              'name': 'Objective Value',
              'type': 'scatter',
              'x': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,
                    18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
                    34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49,
                    50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65,
                    66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81,
                    82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97,
                    98, 99],
              'y': [0.6813926102113866, 0.6769969801905161, 0.681217717770238,
                    0.682546900322968, 0.6811594202898551, 0.6711905511443795,
                    0.6825585598190447, 0.682465283850432, 0.6818939685426796,
                    0.680634742966409, 0.6824769433465085, 0.6824536243543554,
                    0.6781745892942507, 0.6824186458661257, 0.6789557755313815,
                    0.6821971154406706, 0.6801567036272692, 0.683048258654261,
                    0.6825119218347383, 0.6796436857998998, 0.6791539869646834,
                    0.682406986370049, 0.682465283850432, 0.6810078468408596,
                    0.6818473305583733, 0.6825702193151212, 0.6825119218347383,
                    0.6731376869891683, 0.6820338824955985, 0.6824419648582788,
                    0.681299334242774, 0.682406986370049, 0.6826051978033509,
                    0.6823720078818194, 0.6824186458661257, 0.6792356034372194,
                    0.6818589900544498, 0.6753063532594121, 0.6817890330779903,
                    0.6824769433465085, 0.6819989040073688, 0.6824769433465085,
                    0.6823603483857428, 0.6822204344328238, 0.6823486888896662,
                    0.6820805204799049, 0.6825818788111978, 0.6827451117562698,
                    0.6808329543997108, 0.6822554129210535, 0.6677276808096354,
                    0.6824886028425852, 0.6829666421817249, 0.678221227278557,
                    0.6828383877248825, 0.6785943311530076, 0.6819872445112922,
                    0.6799235137057377, 0.682686814275887, 0.6825818788111978,
                    0.6826285167955041, 0.6825002623386617, 0.6827217927641167,
                    0.6828733662131122, 0.6826168572994276, 0.6813576317231569,
                    0.6819989040073688, 0.682908344701342, 0.6817773735819138,
                    0.6821971154406706, 0.6830832371424908, 0.6828383877248825,
                    0.6824186458661257, 0.6815441836603822, 0.6804715100213369,
                    0.682325369897513, 0.6821621369524409, 0.6825235813308149,
                    0.6822554129210535, 0.67960870731167, 0.6830599181503375,
                    0.6823836673778959, 0.6824769433465085, 0.6825119218347383,
                    0.6818589900544498, 0.6818356710622967, 0.6825935383072743,
                    0.6825002623386617, 0.6827101332680402, 0.6825702193151212,
                    0.6824536243543554, 0.6826751547798104, 0.6774400410414262,
                    0.682686814275887, 0.6823486888896662, 0.6828383877248825,
                    0.6828034092366528, 0.6822554129210535, 0.6823720078818194,
                    0.6813226532349271]},
             {'mode': 'lines',
              'name': 'Best Value',
              'type': 'scatter',
              'x': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,
                    18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
                    34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49,
                    50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65,
                    66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81,
                    82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97,
                    98, 99],
              'y': [0.6813926102113866, 0.68139261021138

In [22]:
optuna.visualization.plot_param_importances(study)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'cliponaxis': False,
              'hovertemplate': [max_features (CategoricalDistribution):
                                0.002130241030473181<extra></extra>, criterion
                                (CategoricalDistribution):
                                0.009813921261843168<extra></extra>, n_estimators
                                (IntDistribution):
                                0.010857604327795995<extra></extra>,
                                min_samples_leaf (IntDistribution):
                                0.040664865875593544<extra></extra>,
                                min_samples_split (IntDistribution):
                                0.0467810252557338<extra></extra>, max_depth
                                (IntDistribution):
                                0.8897523422485603<extra></extra>],
              'name': 'Objective Value',
              'orientation': 'h',
              'text': [<0.01, <0.01, 0.01, 0.04, 0.05, 0.89],
              'textposition': 'outside',
              'type': 'bar',
              'x': [0.002130241030473181, 0.009813921261843168,
                    0.010857604327795995, 0.040664865875593544, 0.0467810252557338,
                    0.8897523422485603],
              'y': [max_features, criterion, n_estimators, min_samples_leaf,
                    min_samples_split, max_depth]}],
    'layout': {'template': '...',
               'title': {'text': 'Hyperparameter Importances'},
               'xaxis': {'title': {'text': 'Hyperparameter Importance'}},
               'yaxis': {'title': {'text': 'Hyperparameter'}}}
})

In [23]:
optuna.visualization.plot_slice(study, params = ['n_estimators',
'max_depth',
'min_samples_split',
'min_samples_leaf',
'max_features',
'criterion',])

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'marker': {'color': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,
                                   14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25,
                                   26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37,
                                   38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49,
                                   50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61,
                                   62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73,
                                   74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
                                   86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97,
                                   98, 99],
                         'colorbar': {'title': {'text': 'Trial'}, 'x': 1.0, 'xpad': 40},
                         'colorscale': [[0.0, 'rgb(247,251,255)'], [0.125,
                                        'rgb(222,235,247)'], [0.25,
                                        'rgb(198,219,239)'], [0.375,
                                        'rgb(158,202,225)'], [0.5,
                                        'rgb(107,174,214)'], [0.625,
                                        'rgb(66,146,198)'], [0.75,
                                        'rgb(33,113,181)'], [0.875,
                                        'rgb(8,81,156)'], [1.0, 'rgb(8,48,107)']],
                         'line': {'color': 'Grey', 'width': 0.5},
                         'showscale': True},
              'mode': 'markers',
              'name': 'Feasible Trial',
              'showlegend': False,
              'type': 'scatter',
              'x': [gini, gini, log_loss, entropy, entropy, entropy, log_loss,
                    gini, log_loss, log_loss, log_loss, entropy, entropy, log_loss,
                    entropy, log_loss, entropy, entropy, log_loss, gini, log_loss,
                    entropy, entropy, entropy, entropy, entropy, entropy, entropy,
                    gini, log_loss, gini, entropy, entropy, entropy, entropy,
                    entropy, entropy, gini, log_loss, entropy, log_loss, entropy,
                    entropy, entropy, entropy, entropy, log_loss, log_loss,
                    log_loss, log_loss, log_loss, log_loss, log_loss, log_loss,
                    log_loss, log_loss, log_loss, log_loss, log_loss, log_loss,
                    log_loss, log_loss, log_loss, log_loss, log_loss, log_loss,
                    log_loss, log_loss, log_loss, log_loss, gini, gini, gini, gini,
                    gini, gini, gini, gini, gini, gini, gini, gini, gini, gini,
                    gini, gini, log_loss, gini, log_loss, log_loss, gini, log_loss,
                    log_loss, log_loss, log_loss, log_loss, log_loss, log_loss,
                    log_loss, gini],
              'xaxis': 'x',
              'y': [0.6813926102113866, 0.6769969801905161, 0.681217717770238,
                    0.682546900322968, 0.6811594202898551, 0.6711905511443795,
                    0.6825585598190447, 0.682465283850432, 0.6818939685426796,
                    0.680634742966409, 0.6824769433465085, 0.6824536243543554,
                    0.6781745892942507, 0.6824186458661257, 0.6789557755313815,
                    0.6821971154406706, 0.6801567036272692, 0.683048258654261,
                    0.6825119218347383, 0.6796436857998998, 0.6791539869646834,
                    0.682406986370049, 0.682465283850432, 0.6810078468408596,
                    0.6818473305583733, 0.6825702193151212, 0.6825119218347383,
                    0.6731376869891683, 0.6820338824955985, 0.6824419648582788,
                    0.681299334242774, 0.682406986370049, 0.6826051978033509,
                    0.6823720078818194, 0.6824186458661257, 0.6792356034372194,
                    0.6818589900544498, 0.6753063532594121, 0.6817890330779903,
                    0.6824769433465085, 0.6819989040073688, 0.6824769433465085,
                    0.6823603483857428, 0.6822204344328238, 0.6823486

In [25]:
model = RandomForestClassifier(
        n_estimators=311,
        max_depth=14,
        min_samples_split=23,
        min_samples_leaf=4,
        max_features="log2",
        criterion="gini"
    )
model.fit(X_train_scaled, y_train)


RandomForestClassifier(max_depth=14, max_features='log2', min_samples_leaf=4,
                       min_samples_split=23, n_estimators=311)

In [27]:
model.score(X_test_scaled, y_test)

0.6825935383072743

In [33]:
# save model
import pickle
with open("../models/trained_rf.save", "wb") as f:
    pickle.dump(model, f)